In [ ]:
# prelude
import energnn
from ase.visualize import view
from ase.build import molecule
import torch_geometric.loader as tcg_loader
import torch as tc
from functools import reduce
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params


# Data Preparation
This code only need to be executed at the first time.

In [ ]:
# Download the files that needed.
import energnn as erg
erg.Toolbelt.download_environment_files()

# BasicGNN

In [ ]:
# dataset prepare.
import pickle
from energnn import DatasetAlexandria
wbm = energnn.DatasetWBM()



In [ ]:
#print(alex[60].y)
#print(wbm[880].y)
from energnn import DatasetAlexandria
DatasetAlexandria(load_files=['005', '006', '007', '008', '009'], cutoff=8.0).dump("/home/aylwin/Projects/matbench/tmp/alexset005-009_cutoff8.dump")

## Train model

In [ ]:
# new model
model = energnn.ModelBasicGNNDeeper()

In [ ]:
%%prun
l1loss = tc.nn.L1Loss()
mse_loss = tc.nn.MSELoss()
def train_loss_fn(y, pred):
    # This function will just train on e_above_hull aka y[:, 1]
    return mse_loss(y[:, 1], pred[:, 1])
def test_loss():
    n = 0
    energy_sum, e_above_hull_sum = 0, 0
    num_Acc, n_Acc = 0, 0
    def return_loss(y:tc.Tensor, pred:tc.Tensor)->dict:
        nonlocal n, energy_sum, e_above_hull_sum, num_Acc, n_Acc
        energy = l1loss(y[:, 0], pred[:, 0])
        e_above_hull = l1loss(y[:, 1], pred[:, 1])
        n+=1
        # determine the Acc.  *----XOR---*
        num_Acc += (lambda x, y:(1-(x+y)%2).sum())((y[:, 1]>0).int(), (pred[:, 1]>0).int())
        #print((y[:, 1]>0).int().sum())
        n_Acc += y.shape[0]
        # add energy and e_above_hull_sum
        energy_sum += float(energy)
        e_above_hull_sum += float(e_above_hull)
        return {"avr_energy":energy_sum/n, "avr_e_above_hull":e_above_hull_sum/n, "avr_accuracy":num_Acc/n_Acc}
    return return_loss
for epoch in range(40):
    print(f"Epoch {epoch} ================>")
    print("train:")
    for train_name in ('000-004', '005-009', '010-014'):
        with open(f"/home/aylwin/Projects/matbench/tmp/alexset{train_name}_cutoff8.dump", 'rb') as f:
            train_set = pickle.load(f)
        model, log = mb.tcg_trainer(
            model=model,
            dataset=train_set,
            optimizer=tc.optim.Adam(model.parameters(), lr=0.0001),
            device='cuda',
            epoch=1, 
            batch_size=64,
            num_workers=4,
            loss_fn = train_loss_fn
        )
    print("test:")
    mb.tcg_tester(
        model=model,
        dataset=wbm,
        device='cuda',
        batch_size=64,
        num_workers=4,
        loss_fn=test_loss,
    )


In [ ]:
# test
l1loss = tc.nn.L1Loss()
def test_loss():
    n = 0
    energy_sum, e_above_hull_sum = 0, 0
    num_Acc, n_Acc = 0, 0
    def return_loss(y:tc.Tensor, pred:tc.Tensor)->dict:
        nonlocal n, energy_sum, e_above_hull_sum, num_Acc, n_Acc
        energy = l1loss(y[:, 0], pred[:, 0])
        e_above_hull = l1loss(y[:, 1], pred[:, 1])
        n+=1
        # determine the Acc.  *----XOR---*
        num_Acc += (lambda x, y:(1-(x+y)%2).sum())((y[:, 1]>0).int(), (pred[:, 1]>0).int())
        #print((y[:, 1]>0).int().sum())
        n_Acc += y.shape[0]
        # add energy and e_above_hull_sum
        energy_sum += float(energy)
        e_above_hull_sum += float(e_above_hull)
        return {"avr_energy":energy_sum/n, "avr_e_above_hull":e_above_hull_sum/n, "avr_accuracy":num_Acc/n_Acc}
    return return_loss
energnn.tcg_tester(
    model=model,
    dataset=wbm,
    device='cuda',
    batch_size=256,
    num_workers=32,
    loss_fn=test_loss,
)
energnn.tcg_tester(
    model=model,
    dataset=alex_train,
    device='cuda',
    batch_size=256,
    num_workers=32,
    loss_fn=test_loss,
)
energnn.tcg_tester(
    model=model,
    dataset=alex_test,
    device='cuda',
    batch_size=256,
    num_workers=32,
    loss_fn=test_loss,
)

In [ ]:
# save the module.
import pickle
with open("/home/aylwin/Projects/matbench/tmp/good_model.ptm", "wb") as f:
    pickle.dump(model, f)

In [ ]:
count_parameters(model)

# EnerGNN
---
This model `.forward` will return the total energy for the structure input.

## Train Model

In [5]:
# Here is an example of how to train the EnerGNN model on the Alexandria dataset.
import energnn as erg
import torch as tc

# Load the dataset.
# dataset = erg.datasetAlexandriaNeo(load_files=['000'], cutoff=6.0) # This just run in the first time.
dataset = erg.Toolbelt.pk_load("/Alexandria/000_cutoff6.dump")

# init model
model = erg.EnerG()
l1loss = tc.nn.L1Loss()
model, log = erg.tcg_trainer(
    model=model,
    dataset=dataset,
    optimizer=tc.optim.Adam(model.parameters(), lr=0.0001),
    device='cuda',
    epoch=10, 
    batch_size=64,
    num_workers=4,
    loss_fn = l1loss
)


trainer start...
Epoch 0 ===========>


  0%|          | 0/1563 [00:00<?, ?it/s]/home/aylwin/Projects/matbench/.pixi/envs/default/lib/python3.12/site-packages/torch/nn/modules/loss.py:129: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/aylwin/Projects/matbench/EnerGNN/energnn.py:487: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1769243867349/work/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  sum_loss += float(loss)
100%|█████████▉| 1562/1563 [01:51<00:00, 14.01it/s, avr_loss:804182.429456176]  /home/aylwin/Projects/matbench/.pixi/envs/default/lib/python3.12/site-packages/torch/nn/modules/loss.py:129: UserWar

Epoch 1 ===========>


100%|██████████| 1563/1563 [01:51<00:00, 14.03it/s, avr_loss:35876.97659280281] 


Epoch 2 ===========>


...

Epoch 8 ===========>


100%|██████████| 1563/1563 [01:51<00:00, 13.97it/s, avr_loss:131.60589655621723]


Epoch 9 ===========>


100%|██████████| 1563/1563 [01:52<00:00, 13.94it/s, avr_loss:102.52171038666064]


(EnerG(
   (conv1): NNConv(4, 8, aggr=add, nn=Sequential(
     (0): Linear(in_features=3, out_features=64, bias=True)
     (1): LeakyReLU(negative_slope=0.1)
     (2): Linear(in_features=64, out_features=32, bias=True)
     (3): LeakyReLU(negative_slope=0.1)
   ))
   (conv2): NNConv(8, 64, aggr=add, nn=Sequential(
     (0): Linear(in_features=3, out_features=64, bias=True)
     (1): LeakyReLU(negative_slope=0.1)
     (2): Linear(in_features=64, out_features=512, bias=True)
     (3): LeakyReLU(negative_slope=0.1)
   ))
   (conv3): NNConv(64, 128, aggr=add, nn=Sequential(
     (0): Linear(in_features=3, out_features=64, bias=True)
     (1): LeakyReLU(negative_slope=0.1)
     (2): Linear(in_features=64, out_features=8192, bias=True)
     (3): LeakyReLU(negative_slope=0.1)
   ))
   (fc1): Linear(128, 128, bias=True)
   (fc2): Linear(128, 64, bias=True)
   (fc3): Linear(64, 1, bias=True)
   (leaky): LeakyReLU(negative_slope=0.1)
 ),
 {'avr_loss': [803782.7304450076,
   35876.97659280281,
  

## Get force from structure

In [2]:
# TODO
import energnn as erg
dataset = erg.DatasetWBM()
print(dataset[0])
for i in dataset:
    print(i)
    break

Init the Dataset WBM.


100%|██████████| 256963/256963 [00:38<00:00, 6668.17it/s]


Data(edge_index=[2, 104], y=[1, 2], node_features=[8, 4], edge_weight=[104])
Data(edge_index=[2, 104], y=[1, 2], node_features=[8, 4], edge_weight=[104])
